<a href="https://colab.research.google.com/github/Endalebob/Deep-Learning-Lab/blob/main/DL_Lab_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch

In [20]:
class DenseLayer:
  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.biases

In [21]:
class Activation_ReLU:
  # Forward pass
  def forward(self, inputs):
    self.output = torch.max(torch.tensor(0),inputs)

In [22]:
class Activation_Sigmoid:
  # Forward pass
  def forward(self, inputs):
    self.output = 1 / (1 + torch.exp(-inputs))
    return self.output

In [23]:
class Activation_Softmax:
  # Forward pass
  def forward(self, inputs):
    # Get unnormalized probabilities
    exp_values = torch.exp(inputs - torch.max(inputs, axis=1, keepdim=True).values)
    # Normalize them for each sample
    probabilities = exp_values / torch.sum(exp_values, axis=1, keepdim=True)
    self.output = probabilities

In [24]:
class Accuracy():
  def calculate(self, y_pred, y_true):
    predictions = torch.argmax(y_pred, axis=1)
    if len(y_true.shape) == 2:
      y_true = torch.argmax(y_true, axis=1)
    accuracy = torch.mean((predictions == y_true).float())
    return accuracy

In [25]:
def mean_squared_error(y_true, y_pred):
    # Calculate Mean Squared Error
    loss = torch.mean((y_true - y_pred) ** 2)
    return loss

In [26]:
# Input, Output Data
X = torch.tensor([[0.1, 0.2], [0.4, 0.5], [0.7, 0.8]])  # Example input data with 2 features
y = torch.tensor([[0.3, 0.4], [0.6, 0.7], [0.9, 1.0]])  # Example output data with 2 targets

In [27]:
# Define network architecture
hidden_layer = DenseLayer(2, 4)  # 2 input features, 4 neurons in hidden layer
output_layer = DenseLayer(4, 2)  # 4 neurons in hidden layer, 2 output neurons
activation_sigmoid = Activation_Sigmoid()

In [28]:
# Hyperparameters
learning_rate = 0.01
loss_threshold = 0.0001
epochs = 10000

In [29]:
# Function for forward pass
def forward_pass(x):
    hidden_layer.forward(x)
    hidden_activation = activation_sigmoid.forward(hidden_layer.output)
    output_layer.forward(hidden_activation)
    return output_layer.output

In [30]:
# Function for backpropagation
def back_prop(fp):
    # Calculate the error in the output layer
    output_error = fp - y
    output_delta = output_error  # Gradient of the loss w.r.t. output layer's

    # Backpropagate the output delta to the hidden layer
    hidden_error = torch.matmul(output_delta, output_layer.weights.T)

    # Calculate the derivative of the hidden layer's activation function
    hidden_activation_derivative = activation_sigmoid.forward(hidden_layer.output) * (
        1 - activation_sigmoid.forward(hidden_layer.output)
    )

    # Compute the delta for the hidden layer
    hidden_delta = hidden_error * hidden_activation_derivative  # Gradient of the loss w.r.t. hidden layer's pre-activation values

    # Update output layer weights and biases using gradient descent
    output_layer.weights -= learning_rate * torch.matmul(
        activation_sigmoid.output.T, output_delta
    )  # Update rule for output layer weights
    output_layer.biases -= learning_rate * torch.sum(output_delta, axis=0, keepdim=True)  # Update rule for output layer biases

    # Update hidden layer weights and biases using gradient descent
    hidden_layer.weights -= learning_rate * torch.matmul(X.T, hidden_delta)  # Update rule for hidden layer weights
    hidden_layer.biases -= learning_rate * torch.sum(hidden_delta, axis=0, keepdim=True)  # Update rule for hidden layer biases


In [31]:
# Training loop
for epoch in range(epochs):
    # Forward pass
    predictions = forward_pass(X)

    # Compute loss
    loss = mean_squared_error(y, predictions)

    # Backward pass (Gradient Descent)
    back_prop(predictions)

    if loss < loss_threshold:
        print(f"Training stopped at epoch {epoch + 1} with Loss: {loss.item()}")
        break

Training stopped at epoch 7459 with Loss: 9.992767445510253e-05


In [32]:
# After training, use the model for predictions
final_predictions = forward_pass(X)
print("Final loss:", loss)
print("Final prediction:",final_predictions)
print("Target value:",y)

Final loss: tensor(9.9928e-05)
Final prediction: tensor([[0.3065, 0.4045],
        [0.6097, 0.7097],
        [0.8860, 0.9877]])
Target value: tensor([[0.3000, 0.4000],
        [0.6000, 0.7000],
        [0.9000, 1.0000]])
